## Problem-2 Part-1 Indic NER Model Finetuning:

In [31]:
!pip install transformers
!pip install torch
!pip install datasets
!pip install scikit-learn
!pip install accelerate

In [32]:
# from google.colab import drive
# drive.mount('/content/drive')

### Importing required libraries-

In [33]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments,DataCollatorForTokenClassification ,AutoModelForSequenceClassification
import torch
from datasets import Dataset
import json
import time
from sklearn.metrics import f1_score
import numpy as np

### Importing Tokenizers and Models from ai4bharat:

In [34]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
import numpy as np

config = AutoConfig.from_pretrained('ai4bharat/indic-bert', num_labels=7, finetuning_task='ner')
tokenizer_bert = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model_bert = AutoModelForTokenClassification.from_pretrained('ai4bharat/indic-bert',num_labels=7)

Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
# Move the model to the GPU (use 'cuda' or 'cuda:0' for Colab)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model_bert = model_bert.to(device)

### Loading dataset:

In [36]:
%%time
# !pip install -U datasets==2.15.0
from datasets import load_dataset
# path = '/kaggle/input/dataset/hi_IndicNER_v1.0'
# # # path = '/content/drive/MyDrive/hi_IndicNER_v1.0'
# # path = '/prashantmishraa/dataset/hi_IndicNER_v1.0'
# data = load_dataset(path)

# from datasets import load_dataset
data = load_dataset('ai4bharat/naamapadam', 'hi')
print(data)

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 985787
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 867
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 13460
    })
})
CPU times: user 83.4 ms, sys: 8.07 ms, total: 91.5 ms
Wall time: 675 ms


### Selecting random data points to finetune the Model:

In [37]:
train_dataset = data['train'].shuffle(seed=42).select(range(50000))       #985787
validation_dataset = data['validation'].shuffle(seed=42).select(range(13000)) #13460
test_dataset = data['test'].shuffle(seed=42).select(range(850))

In [38]:
print(data)

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 985787
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 867
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 13460
    })
})


### Function to Tokenize data and assign corresponding labels with data:

In [40]:
padding = "max_length"
text_column_name = 'tokens'
label_column_name = 'ner_tags'

# Map NER labels to numerical values
label_mapping = {'O': 0, 'B-ORG': 1, 'I-ORG': 2, 'B-PER': 3, 'I-PER': 4, 'B-LOC': 5, 'I-LOC': 6}

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer_bert(
        examples[text_column_name],
        truncation=True,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label))
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

### Mapping the data for compatibility with the Trainer class:

In [41]:
%%time
tokenized_train_dataset =train_dataset.map(tokenize_and_align_labels,batched=True,load_from_cache_file=True,num_proc=10)
tokenized_validation_dataset =validation_dataset.map(tokenize_and_align_labels,batched=True,load_from_cache_file=True,num_proc=10)
tokenized_test_dataset =test_dataset.map(tokenize_and_align_labels,batched=True,load_from_cache_file=True,num_proc=10)

#0:   0%|          | 0/5 [00:00<?, ?ba/s]

#1:   0%|          | 0/5 [00:00<?, ?ba/s]

#2:   0%|          | 0/5 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


...

#7:   0%|          | 0/1 [00:00<?, ?ba/s]

#8:   0%|          | 0/1 [00:00<?, ?ba/s]

#9:   0%|          | 0/1 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


CPU times: user 1.52 s, sys: 919 ms, total: 2.43 s
Wall time: 12.1 s


In [42]:
print(tokenized_train_dataset)

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 50000
})


### Function to compute matric used to validate the performance of Model and Evaluate function to eval any dataset:

In [43]:
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

def compute_metrics(p):
    predictions, labels = p.predictions, p.label_ids

    # Flatten the predictions and labels
    predictions_flat = np.argmax(predictions, axis=2).flatten()
    labels_flat = labels.flatten()

    # Remove occurrences of -100 from both predictions and labels
    mask = (labels_flat != -100)
    predictions_filtered = predictions_flat[mask]
    labels_filtered = labels_flat[mask]

    # Calculate macro F1 score
    f1 = f1_score(labels_filtered, predictions_filtered, average="macro")

    # Calculate precision, recall, and accuracy with zero_division parameter
    precision = precision_score(labels_filtered, predictions_filtered, average="macro", zero_division=1)
    recall = recall_score(labels_filtered, predictions_filtered, average="macro")
    accuracy = accuracy_score(labels_filtered, predictions_filtered)

    return {
        "macro_f1": f1,
        "precision": precision,
        "recall": recall,
        "accuracy": accuracy
    }


def evaluate_model(trainer, dataset):
    results = trainer.evaluate(dataset)
    return results

### Specifing Training args and hyperparameters:

In [44]:
%%time
import numpy as np
data_collator = DataCollatorForTokenClassification(tokenizer_bert)
# function for Fine-tuning BERTner
def fine_tune_indic_bert(train_dataset, validation_dataset):
    training_args = TrainingArguments(
        output_dir="./indic_bert_fine_tuned",
        num_train_epochs=5,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=16,
        logging_dir="./indicBERT_ner_logs",
        evaluation_strategy="epoch",
        save_total_limit=2,
        save_strategy = "epoch",
        load_best_model_at_end=True,
        metric_for_best_model="macro_f1",
        gradient_accumulation_steps = 2,
    )

    trainer = Trainer(
        model=model_bert,
        args=training_args,
        train_dataset=train_dataset,
        data_collator = data_collator,
        eval_dataset=validation_dataset,
        compute_metrics=compute_metrics,  # Use the custom metrics function
    )

    trainer.train()
    return trainer

CPU times: user 41 µs, sys: 0 ns, total: 41 µs
Wall time: 44.3 µs


### Finetuning the Model with training data and val. data:

In [45]:
%%time
import os
import torch
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
torch.cuda.empty_cache()
# starting the finetuning of IndicBERT
indic_bert_trainer = fine_tune_indic_bert(tokenized_train_dataset, tokenized_validation_dataset)

Epoch,Training Loss,Validation Loss,Macro F1,Precision,Recall,Accuracy
0,0.404500,0.274831,0.714112,0.773329,0.675788,0.916120
2,0.220500,0.238710,0.759230,0.793759,0.730334,0.927057
4,0.172400,0.243337,0.761895,0.783368,0.742938,0.926512


CPU times: user 42min 59s, sys: 6.78 s, total: 43min 6s
Wall time: 43min 3s


### Model performance(macro f1-score) on test data:

In [46]:
indic_bert_results = evaluate_model(indic_bert_trainer, tokenized_test_dataset)

for metric in indic_bert_results:
    print(metric,":  ",indic_bert_results[metric])

eval_loss :   0.1937623918056488
eval_macro_f1 :   0.7916347848413824
eval_precision :   0.8054039653315705
eval_recall :   0.7791856963796026
eval_accuracy :   0.9406164771265373
eval_runtime :   2.6616
eval_samples_per_second :   319.362
eval_steps_per_second :   20.289
epoch :   5.0


### macro f1-score on training data:

In [47]:
indic_bert_results = evaluate_model(indic_bert_trainer, tokenized_train_dataset)

for metric in indic_bert_results:
    print(metric,":  ",indic_bert_results[metric])

eval_loss :   0.15485289692878723
eval_macro_f1 :   0.8466177864011207
eval_precision :   0.8662768845763477
eval_recall :   0.829012986248066
eval_accuracy :   0.9525205694839322
eval_runtime :   168.9596
eval_samples_per_second :   295.929
eval_steps_per_second :   18.496
epoch :   5.0


### macro f1-score on validation data:

In [48]:
indic_bert_results = evaluate_model(indic_bert_trainer, tokenized_validation_dataset)

for metric in indic_bert_results:
    print(metric,":  ",indic_bert_results[metric])

eval_loss :   0.24333663284778595
eval_macro_f1 :   0.761895419919979
eval_precision :   0.7833682740779817
eval_recall :   0.7429380062448184
eval_accuracy :   0.926511947809625
eval_runtime :   43.868
eval_samples_per_second :   296.343
eval_steps_per_second :   18.533
epoch :   5.0


**To view logs of any epoch..** uncomment below cell

In [49]:
train_metrics = indic_bert_trainer.state.log_history
epoch_num = len(train_metrics)-1
from pprint import pprint
pprint(train_metrics[epoch_num])

{'epoch': 5.0,
 'eval_accuracy': 0.926511947809625,
 'eval_loss': 0.24333663284778595,
 'eval_macro_f1': 0.761895419919979,
 'eval_precision': 0.7833682740779817,
 'eval_recall': 0.7429380062448184,
 'eval_runtime': 43.868,
 'eval_samples_per_second': 296.343,
 'eval_steps_per_second': 18.533,
 'step': 3905}


### Saving the model, to be used in 4th Problem:

In [50]:
indic_bert_trainer.save_model("finetunedBERT_ner_model")
tokenizer_bert.save_pretrained("finetunedBERT_ner_model")

('finetunedBERT_ner_model/tokenizer_config.json',
 'finetunedBERT_ner_model/special_tokens_map.json',
 'finetunedBERT_ner_model/spiece.model',
 'finetunedBERT_ner_model/added_tokens.json',
 'finetunedBERT_ner_model/tokenizer.json')